# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f763c5b8e50>
├── 'a' --> tensor([[ 1.7684,  0.6979, -0.9201],
│                   [-0.5249, -1.0979, -0.4504]])
└── 'x' --> <FastTreeValue 0x7f763c557b80>
    └── 'c' --> tensor([[-0.3935,  1.0797,  2.2171, -0.2648],
                        [ 1.0158,  1.9922,  1.3555, -0.0059],
                        [-0.8977,  0.6875,  0.7914,  0.8611]])

In [4]:
t.a

tensor([[ 1.7684,  0.6979, -0.9201],
        [-0.5249, -1.0979, -0.4504]])

In [5]:
%timeit t.a

69.4 ns ± 0.0165 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f763c5b8e50>
├── 'a' --> tensor([[-0.6247,  1.1244, -0.9386],
│                   [-0.0660, -1.7112, -2.0395]])
└── 'x' --> <FastTreeValue 0x7f763c557b80>
    └── 'c' --> tensor([[-0.3935,  1.0797,  2.2171, -0.2648],
                        [ 1.0158,  1.9922,  1.3555, -0.0059],
                        [-0.8977,  0.6875,  0.7914,  0.8611]])

In [7]:
%timeit t.a = new_value

59.6 ns ± 0.0423 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.7684,  0.6979, -0.9201],
               [-0.5249, -1.0979, -0.4504]]),
    x: Batch(
           c: tensor([[-0.3935,  1.0797,  2.2171, -0.2648],
                      [ 1.0158,  1.9922,  1.3555, -0.0059],
                      [-0.8977,  0.6875,  0.7914,  0.8611]]),
       ),
)

In [10]:
b.a

tensor([[ 1.7684,  0.6979, -0.9201],
        [-0.5249, -1.0979, -0.4504]])

In [11]:
%timeit b.a

57.3 ns ± 0.0112 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.4352,  0.3558,  1.5319],
               [ 0.7519, -0.1134, -0.3102]]),
    x: Batch(
           c: tensor([[-0.3935,  1.0797,  2.2171, -0.2648],
                      [ 1.0158,  1.9922,  1.3555, -0.0059],
                      [-0.8977,  0.6875,  0.7914,  0.8611]]),
       ),
)

In [13]:
%timeit b.a = new_value

488 ns ± 0.191 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

840 ns ± 0.781 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.7 µs ± 29.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 417 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 628 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f763c552220>
├── 'a' --> tensor([[[ 1.7684,  0.6979, -0.9201],
│                    [-0.5249, -1.0979, -0.4504]],
│           
│                   [[ 1.7684,  0.6979, -0.9201],
│                    [-0.5249, -1.0979, -0.4504]],
│           
│                   [[ 1.7684,  0.6979, -0.9201],
│                    [-0.5249, -1.0979, -0.4504]],
│           
│                   [[ 1.7684,  0.6979, -0.9201],
│                    [-0.5249, -1.0979, -0.4504]],
│           
│                   [[ 1.7684,  0.6979, -0.9201],
│                    [-0.5249, -1.0979, -0.4504]],
│           
│                   [[ 1.7684,  0.6979, -0.9201],
│                    [-0.5249, -1.0979, -0.4504]],
│           
│                   [[ 1.7684,  0.6979, -0.9201],
│                    [-0.5249, -1.0979, -0.4504]],
│           
│                   [[ 1.7684,  0.6979, -0.9201],
│                    [-0.5249, -1.0979, -0.4504]]])
└── 'x' --> <FastTreeValue 0x7f763c552250>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.8 µs ± 25.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f763c557850>
├── 'a' --> tensor([[ 1.7684,  0.6979, -0.9201],
│                   [-0.5249, -1.0979, -0.4504],
│                   [ 1.7684,  0.6979, -0.9201],
│                   [-0.5249, -1.0979, -0.4504],
│                   [ 1.7684,  0.6979, -0.9201],
│                   [-0.5249, -1.0979, -0.4504],
│                   [ 1.7684,  0.6979, -0.9201],
│                   [-0.5249, -1.0979, -0.4504],
│                   [ 1.7684,  0.6979, -0.9201],
│                   [-0.5249, -1.0979, -0.4504],
│                   [ 1.7684,  0.6979, -0.9201],
│                   [-0.5249, -1.0979, -0.4504],
│                   [ 1.7684,  0.6979, -0.9201],
│                   [-0.5249, -1.0979, -0.4504],
│                   [ 1.7684,  0.6979, -0.9201],
│                   [-0.5249, -1.0979, -0.4504]])
└── 'x' --> <FastTreeValue 0x7f763c5578e0>
    └── 'c' --> tensor([[-0.3935,  1.0797,  2.2171, -0.2648],
                        [ 1.0158,  1.9922,  1.3555, -0.0059],
                 

In [23]:
%timeit t_cat(trees)

30.6 µs ± 66.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.5 µs ± 181 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 1.7684,  0.6979, -0.9201],
                [-0.5249, -1.0979, -0.4504]],
       
               [[ 1.7684,  0.6979, -0.9201],
                [-0.5249, -1.0979, -0.4504]],
       
               [[ 1.7684,  0.6979, -0.9201],
                [-0.5249, -1.0979, -0.4504]],
       
               [[ 1.7684,  0.6979, -0.9201],
                [-0.5249, -1.0979, -0.4504]],
       
               [[ 1.7684,  0.6979, -0.9201],
                [-0.5249, -1.0979, -0.4504]],
       
               [[ 1.7684,  0.6979, -0.9201],
                [-0.5249, -1.0979, -0.4504]],
       
               [[ 1.7684,  0.6979, -0.9201],
                [-0.5249, -1.0979, -0.4504]],
       
               [[ 1.7684,  0.6979, -0.9201],
                [-0.5249, -1.0979, -0.4504]]]),
    x: Batch(
           c: tensor([[[-0.3935,  1.0797,  2.2171, -0.2648],
                       [ 1.0158,  1.9922,  1.3555, -0.0059],
                       [-0.8977,  0.6875,  0.7914,  0.8611]],
         

In [26]:
%timeit Batch.stack(batches)

79.1 µs ± 253 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 1.7684,  0.6979, -0.9201],
               [-0.5249, -1.0979, -0.4504],
               [ 1.7684,  0.6979, -0.9201],
               [-0.5249, -1.0979, -0.4504],
               [ 1.7684,  0.6979, -0.9201],
               [-0.5249, -1.0979, -0.4504],
               [ 1.7684,  0.6979, -0.9201],
               [-0.5249, -1.0979, -0.4504],
               [ 1.7684,  0.6979, -0.9201],
               [-0.5249, -1.0979, -0.4504],
               [ 1.7684,  0.6979, -0.9201],
               [-0.5249, -1.0979, -0.4504],
               [ 1.7684,  0.6979, -0.9201],
               [-0.5249, -1.0979, -0.4504],
               [ 1.7684,  0.6979, -0.9201],
               [-0.5249, -1.0979, -0.4504]]),
    x: Batch(
           c: tensor([[-0.3935,  1.0797,  2.2171, -0.2648],
                      [ 1.0158,  1.9922,  1.3555, -0.0059],
                      [-0.8977,  0.6875,  0.7914,  0.8611],
                      [-0.3935,  1.0797,  2.2171, -0.2648],
                      [ 1.0158,  

In [28]:
%timeit Batch.cat(batches)

142 µs ± 363 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

312 µs ± 6.39 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
